In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [3]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

In [4]:
weights

<tf.Variable 'Variable:0' shape=(4,) dtype=float32_ref>

In [5]:
chosen_action

<tf.Tensor 'ArgMax:0' shape=() dtype=int64>

In [6]:
#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [12]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
            print(resp, ww)
        i+=1
print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [1. 0. 0. 0.]
[1.] [1. 1. 1. 1.]
Running reward for the 4 bandits: [ 0.  0.  0. 39.]
[1.037322] [1.000005  1.        1.0000011 1.037322 ]
Running reward for the 4 bandits: [ 1.  0. -1. 87.]
[1.0826261] [1.001005  1.        0.9990011 1.0826261]
Running reward for the 4 bandits: [  0.  -1.  -1. 133.]
[1.12433] [1.0000061 0.999     0.9990021 1.12433  ]
Running reward for the 4 bandits: [ -1.  -1.  -2. 181.]
[1.1662564] [0.9990061 0.999     0.9980011 1.1662564]
Running reward for the 4 bandits: [  0.   0.  -2. 225.]
[1.2034054] [1.000008  1.0000011 0.9980021 1.2034054]
Running reward for the 4 bandits: [ -1.  -1.  -2. 273.]
[1.2426654] [0.999008  0.9990011 0.9980021 1.2426654]
Running reward for the 4 bandits: [ -1.  -1.  -3. 322.]
[1.281502] [0.999008  0.9990011 0.9970001 1.281502 ]
Running reward for the 4 bandits: [ -1.  -1.  -3. 370.]
[1.3184366] [0.99900895 0.9990011  0.9970001  1.3184366 ]
Running reward for the 4 bandits: [ -1.  -2.  -3. 415.]
[1.35